# Automatic regridding of ECMWF data to match CESM 

The purpose of this notebook is to automate regirdding of two datasets based on the one with the lowest resolution.

In [1]:
import xarray as xr
import pandas as pd
import numpy as np

from spharm import Spharmt, regrid

Below are a set of functions useful for the regridding

In [2]:
def regrid_field(field, lat, lon, lat_new, lon_new):
    nlat_old, nlon_old = np.size(lat), np.size(lon)
    nlat_new, nlon_new = np.size(lat_new), np.size(lon_new)
    spec_old = Spharmt(nlon_old, nlat_old, gridtype='regular', legfunc='computed')
    spec_new = Spharmt(nlon_new, nlat_new, gridtype='regular', legfunc='computed')

    field_new = []
    for field_old in field:
        regridded_field =  regrid(spec_old, spec_new, field_old, ntrunc=None, smooth=None)
        field_new.append(regridded_field)

    field_new = np.array(field_new)
    return field_new

## Load the data

In [7]:
model = ['ECMWF','CESM']
ds_ecmwf=xr.open_dataset('ECMWF_12month.nc')
ds_cesm=xr.open_dataset('CESM_12month.nc')

ECMWF handles decreasing latitude. Reverse for comparison with CESM

In [9]:
#Inverse the latitude information in that file
ds_ecmwf = ds_ecmwf.reindex(latitude=list(reversed(ds_ecmwf.latitude)))

## Regridding

The next two cells perform the actual regridding operations for the two variables of interest.

In [10]:
TS_regrid = regrid_field(ds_ecmwf['t2m'].values,
                        ds_ecmwf['latitude'].values,
                        ds_ecmwf['longitude'].values,
                        ds_cesm['lat'].values,
                        ds_cesm['lon'].values)

In [11]:
tp_regrid = regrid_field(ds_ecmwf['tp'].values,
                        ds_ecmwf['latitude'].values,
                        ds_ecmwf['longitude'].values,
                        ds_cesm['lat'].values,
                        ds_cesm['lon'].values)

## Repack into a netcdf

In [12]:
time = ds_ecmwf['time']
latitude = ds_cesm['lat']
longitude = ds_cesm['lon']

t2m = xr.DataArray(TS_regrid,coords=[time,latitude,longitude],dims=["time","latitude","longitude"])
t2m.attrs=ds_ecmwf['t2m'].attrs

In [13]:
tp = xr.DataArray(tp_regrid,coords=[time,latitude,longitude],dims=["time","latitude","longitude"])
tp.attrs=ds_ecmwf['tp'].attrs

In [15]:
# Pack into a new dataset
ds = t2m.to_dataset(name='t2m')
ds['tp'] = tp

ds.attrs = ds_ecmwf.attrs
ds.attrs['description'] = "This dataset was regridded to CESM grid"

#Inverse the latitude to go back to ECMWF standard
ds=ds.sortby('latitude', ascending=False)

In [16]:
ds.to_netcdf('ECMWF_regridded_12month.nc')